# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1

Import dependencies

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

The request library is used to load the HTML code from wikipedia and then BeautifulSoup is used to load the data into a dataframe

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html = requests.get(url).text

soup = BeautifulSoup(html, 'html.parser')

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
toronto_neighborhoods = pd.DataFrame(columns=column_names)

data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

# read all rows and store them into the dataframe
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if cols != []:
        toronto_neighborhoods = toronto_neighborhoods.append({'Borough': cols[1],
                                          'Neighborhood': cols[2],
                                          'PostalCode': cols[0]}, ignore_index=True)

Only the rows with Borough assigned are used, and the rows with Neighborhood as "Not assigned" are replaced by the name of the Borough

In [21]:
toronto_neighborhoods = toronto_neighborhoods.loc[(toronto_neighborhoods['Borough'] != 'Not assigned')]
toronto_neighborhoods.loc[(toronto_neighborhoods['Neighborhood'] == 'Not assigned'), 'Neighborhood'] = toronto_neighborhoods.loc[(toronto_neighborhoods['Neighborhood'] == 'Not assigned')]['Borough'].values

Grouping all neighbourhoods with the same postal code

In [18]:
toronto_neighborhoods = toronto_neighborhoods.groupby(['PostalCode', 'Borough']).Neighborhood.agg([('Neighborhood', ', '.join)])

In [20]:
toronto_neighborhoods.shape


(210, 3)

## Part 2